In [14]:
from dataclasses import dataclass
import polars as pl
import pandas as pd
import numpy as np

from numba import njit, prange, float64
import pickle
import time
import re
import copy

import numpy as np
import xgboost as xgb

from typing import Tuple, List, Dict, Any, Union, Literal
from scipy.stats import rankdata, norm
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
from sklearn.feature_selection import mutual_info_regression

warnings.filterwarnings("ignore")


from CONFIG import CONFIG
from PREPROCESSOR_V2 import PREPROCESSOR
from FEATURE_ENGINEERING_V2 import FEATURE_ENGINEERING
from DATASET import SequentialDataset
from ENSEMBLE_NN import ENSEMBLE_NN
from NN import NN
from LOSS import CombinedICIRLoss

import torch
from torch.utils.data import Dataset, DataLoader

# import cuml
# from cuml.explainer import TreeExplainer


def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start:.4f} seconds")
        return result

    return wrapper

In [15]:
def gaussian_rank_transform(arr: np.array):
    # n_samples, n_targets = arr.shape
    transformed_targets = np.full_like(arr, np.nan)

    for i, row in enumerate(arr):
        # Find valid (non-NaN) assets for this timestep
        valid_mask = ~np.isnan(row)
        valid_arr = row[valid_mask]
        ranks = rankdata(valid_arr, method="average")
        percentile_ranks = (ranks - 0.5) / (len(ranks))
        percentile_ranks = np.clip(percentile_ranks, 1e-8, 1 - 1e-8)
        gaussian_values = norm.ppf(percentile_ranks)
        transformed_targets[i, valid_mask] = gaussian_values
    return transformed_targets

In [16]:
class RankCorrelationLoss:
    """Custom loss function for maximizing rank correlation Sharpe ratio."""

    def __init__(self):
        self.num_targets = CONFIG.NUM_TARGET_COLUMNS

    def _compute_rank_correlation_batch(self, y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
        """Compute rank correlation for each sample (date)."""
        y_pred = y_pred.reshape(-1, self.num_targets)
        corr = []
        for i in range(y_pred.shape[0]):
            y_pred_rank = rankdata(y_pred[i], method="average")
            y_true_rank = rankdata(y_true[i], method="average")

            corr.append(np.corrcoef(y_pred_rank, y_true_rank)[0, 1])

        return corr


def rank_correlation_sharpe_eval(y_pred: np.ndarray, y_true: np.ndarray, n_targets: int = 424) -> float:
    """Evaluation function for both XGBoost and LightGBM."""
    loss_fn = RankCorrelationLoss()
    correlations = loss_fn._compute_rank_correlation_batch(y_pred, y_true)

    mean_corr = np.mean(correlations)
    std_corr = np.std(correlations)

    if std_corr == 0:
        return 0.0

    return -mean_corr / std_corr


In [17]:
class FeatureSelector:
    def __init__(self, train_x, train_y, n_targets: int = 424):
        self.train_x = train_x  # drop date
        self.train_y = train_y
        self.n_targets = n_targets
        self.loss_fn = RankCorrelationLoss()
        self.keep_features = None
        self.total_features = train_x.columns.__len__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    @timer
    def basic_filters(self):
        train_x_filter_1 = self.train_x.select([col for col in self.train_x.columns if self.train_x[col].var() > 1e-3])
        train_x_filter_2 = train_x_filter_1.select(
            [col for col in train_x_filter_1.columns if train_x_filter_1[col].value_counts()["count"].max() / len(train_x_filter_1) < 0.80]
        )
        train_x_filter_3 = train_x_filter_2.select([col for col in train_x_filter_2.columns if train_x_filter_2[col].n_unique() > 2])

        print(f"After Basic Filter: {train_x_filter_3.columns.__len__()} / {self.total_features}")

        return train_x_filter_3

    @timer
    def run_correlation(self, x: torch.Tensor, y: torch.Tensor, names: list) -> list:
        """
        Memory-optimized version using chunked processing
        Best for when you have enough GPU memory

        Args:
            x: Tensor
            y: Tensor

        Returns:
            correlations: Tensor
        """
        N, D1 = x.shape
        N2, D2 = y.shape
        assert N == N2

        device = x.device

        # Handle NaNs by masking
        x_valid = ~torch.isnan(x)
        y_valid = ~torch.isnan(y)

        # Convert NaNs to 0 for computation
        x_clean = torch.where(x_valid, x, 0.0)
        y_clean = torch.where(y_valid, y, 0.0)

        # Compute valid sample counts for each pair efficiently
        # This is the memory bottleneck, so we chunk it
        chunk_size = 500  # Adjust based on GPU memory
        correlations = torch.zeros(D1, D2, device=device)

        for i in range(0, D1, chunk_size):
            end_i = min(i + chunk_size, D1)

            # Get chunk
            x_chunk = x_clean[:, i:end_i]  # (N, chunk_size)
            x_valid_chunk = x_valid[:, i:end_i]  # (N, chunk_size)

            # Compute valid sample matrix for this chunk
            valid_matrix = x_valid_chunk.unsqueeze(2) & y_valid.unsqueeze(1)  # (N, chunk_size, D2)
            n_valid = valid_matrix.sum(dim=0).float()  # (chunk_size, D2)

            # Sufficient samples mask
            sufficient = n_valid >= 10

            if sufficient.any():
                # Compute means over valid samples
                x_sum = (x_chunk.unsqueeze(2) * valid_matrix).sum(dim=0)  # (chunk_size, D2)
                y_sum = (y_clean.unsqueeze(1) * valid_matrix).sum(dim=0)  # (chunk_size, D2)

                x_mean = x_sum / (n_valid + 1e-10)
                y_mean = y_sum / (n_valid + 1e-10)

                # Center data
                x_centered = (x_chunk.unsqueeze(2) - x_mean.unsqueeze(0)) * valid_matrix  # (N, chunk_size, D2)
                y_centered = (y_clean.unsqueeze(1) - y_mean.unsqueeze(0)) * valid_matrix  # (N, chunk_size, D2)

                # Compute correlation
                numerator = (x_centered * y_centered).sum(dim=0)
                x_var = (x_centered**2).sum(dim=0)
                y_var = (y_centered**2).sum(dim=0)

                denominator = torch.sqrt(x_var * y_var) + 1e-10
                chunk_corr = numerator / denominator

                # Apply sufficient samples mask
                chunk_corr = torch.where(sufficient, chunk_corr, 0.0)
                correlations[i:end_i] = torch.abs(chunk_corr)

        self.correlations = correlations.cpu().numpy()

    @timer
    def run_biserial_correlation(self, x: torch.Tensor, y: torch.Tensor, names: list) -> list:
        """
        Compute biserial correlations between binary x and continuous y.

        Args:
            x: Binary tensor (N, D1)
            y: Continuous tensor (N, D2)
            names: Unused, for compatibility

        Returns:
            correlations: (D1, D2) Tensor of biserial correlation coefficients
        """
        N, D1 = x.shape
        N2, D2 = y.shape
        assert N == N2

        device = x.device

        # Mask NaNs
        x_valid = ~torch.isnan(x)
        y_valid = ~torch.isnan(y)

        # Replace NaNs with zeros for safe computation
        x_clean = torch.where(x_valid, x, 0.0)
        y_clean = torch.where(y_valid, y, 0.0)

        # Output tensor
        correlations = torch.zeros(D1, D2, device=device)

        chunk_size = 500

        for i in range(0, D1, chunk_size):
            end_i = min(i + chunk_size, D1)

            x_chunk = x_clean[:, i:end_i]  # (N, chunk)
            x_valid_chunk = x_valid[:, i:end_i]  # (N, chunk)

            valid_matrix = x_valid_chunk.unsqueeze(2) & y_valid.unsqueeze(1)  # (N, chunk, D2)
            n_valid = valid_matrix.sum(dim=0).float()  # (chunk, D2)
            sufficient = n_valid >= 10

            if sufficient.any():
                x_bin = x_chunk.unsqueeze(2)  # (N, chunk, 1)
                y_vals = y_clean.unsqueeze(1)  # (N, 1, D2)

                # Get masks for x == 1 and x == 0
                x_eq_1 = (x_bin == 1.0) & valid_matrix  # (N, chunk, D2)
                x_eq_0 = (x_bin == 0.0) & valid_matrix

                # Counts
                n1 = x_eq_1.sum(dim=0).float()
                n0 = x_eq_0.sum(dim=0).float()
                total_n = n1 + n0 + 1e-10  # avoid div by zero

                p = n1 / total_n
                q = 1 - p

                # Means for y where x == 1 and x == 0
                mean_y1 = (y_vals * x_eq_1).sum(dim=0) / (n1 + 1e-10)
                mean_y0 = (y_vals * x_eq_0).sum(dim=0) / (n0 + 1e-10)

                # Overall std of y (on valid entries)
                y_mean = (y_vals * valid_matrix).sum(dim=0) / (n_valid + 1e-10)
                y_centered = (y_vals - y_mean) * valid_matrix
                y_std = torch.sqrt((y_centered**2).sum(dim=0) / (n_valid + 1e-10))

                # Compute biserial correlation
                mean_diff = mean_y1 - mean_y0
                r_bis = (mean_diff / (y_std + 1e-10)) * torch.sqrt(p * q)

                # Mask insufficient pairs
                r_bis = torch.where(sufficient, r_bis, torch.tensor(0.0, device=device))

                correlations[i:end_i] = torch.abs(r_bis)

        self.biserial_correlation = correlations.cpu().numpy()

    def run_selection(self):
        # filtered = self.basic_filters()
        binary_cols = [
            col for col in self.train_x.columns if self.train_x[col].max() == 1 and self.train_x[col].min() == 0 and col != CONFIG.DATE_COL
        ]
        train_x_arr = self.train_x.drop(binary_cols).to_numpy()
        train_x_binary_arr = self.train_x.select(binary_cols).to_numpy()
        train_y_arr = self.train_y.to_numpy()
        x = torch.tensor(train_x_arr, device="cuda")
        binary_x = torch.tensor(train_x_binary_arr, device="cuda")
        y = torch.tensor(train_y_arr, device="cuda")
        self.run_correlation(x=x, y=y, names=self.train_x.columns)
        self.run_biserial_correlation(x=binary_x, y=y, names=binary_cols)

        corrs_df = pl.concat(
            [
                pl.DataFrame(self.correlations.mean(axis=1), schema=["corr_mean"]).with_columns(
                    pl.Series(name="feature", values=self.train_x.drop(binary_cols).columns)
                ),
                pl.DataFrame(self.correlations.std(axis=1), schema=["corr_std"]),
            ],
            how="horizontal",
        )

        keep_features = (
            corrs_df.drop_nans()
            .filter((pl.col("corr_mean") >= corrs_df["corr_mean"].median() * 1.05) & (pl.col("corr_std") <= corrs_df["corr_std"].median() * 0.95))
            .sort(by="corr_std")
            .select("feature")
            .to_series()
            # .head(100)
            .to_list()
        )

        print(f"After correlation, keeping: {keep_features.__len__()} features")

        # MI_scores = self.run_MI(x=np.nan_to_num(self.train_x.select(keep_features).to_numpy()), y=np.nan_to_num(train_y_arr).T)
        # return MI_scores, keep_features
        return self.correlations, self.biserial_correlation, keep_features, binary_cols

In [18]:
# --- Prepare DataLoader ---
# Create the dataset

train_x = pl.scan_csv(CONFIG.TRAIN_X_PATH).drop(CONFIG.DROP_COL)
train_x = PREPROCESSOR(df=train_x)
train_x = train_x.clean()

features = FEATURE_ENGINEERING(df=train_x)
train_x: pl.DataFrame = features.create_market_features()

train_y = pl.scan_csv(CONFIG.TRAIN_Y_PATH)

curr_y = (
    train_y.with_columns([pl.col(CONFIG.LAGS[f"lag{i}"]).exclude(CONFIG.DATE_COL).shift(i + 1) for i in range(1, 5)])
    .with_columns(pl.all().exclude(CONFIG.DATE_COL).shift())
    .filter((pl.col(CONFIG.DATE_COL).is_in(train_x.select(CONFIG.DATE_COL).to_series())))
    .collect()
    .fill_null(0)
    .lazy()
)

y_feat = FEATURE_ENGINEERING(df=curr_y)
ys = y_feat.create_Y_market_features()

train_x = train_x.join(ys, on=CONFIG.DATE_COL).fill_nan(0)


train_y = train_y.filter((pl.col(CONFIG.DATE_COL).is_in(train_x.select(CONFIG.DATE_COL).to_series()))).collect()
train_x = (
    train_x.with_columns([pl.when(pl.col(col).is_infinite()).then(0.0).otherwise(pl.col(col)).alias(col) for col in train_x.columns])
    .with_columns(pl.all().shrink_dtype())
    .filter(pl.col(CONFIG.DATE_COL).is_in(train_y.select(CONFIG.DATE_COL).to_series()))
    .with_columns(pl.col(CONFIG.DATE_COL).cast(pl.Int64))
)

retrain_x = train_x.with_columns(pl.all().exclude(CONFIG.DATE_COL).shift(5))
retrain_y = train_y.filter((pl.col(CONFIG.DATE_COL).is_in(train_x.select(CONFIG.DATE_COL).to_series()))).with_columns(
    pl.all().exclude(CONFIG.DATE_COL).shift(5)
)

train_y_arr = train_y.drop(CONFIG.DATE_COL).to_numpy()

train_y = pl.DataFrame(gaussian_rank_transform(train_y_arr), schema=train_y.drop(CONFIG.DATE_COL).columns).insert_column(
    0, train_y.select(CONFIG.DATE_COL).to_series()
)


# pl.DataFrame(
#     (train_y_arr - np.nanmean(train_y_arr, axis=1).reshape(train_y_arr.shape[0], -1))
#     / np.nanstd(train_y_arr, axis=1).reshape(train_y_arr.shape[0], -1),
#     schema=train_y.drop(CONFIG.DATE_COL).columns,
# ).insert_column(0, train_y.select(CONFIG.DATE_COL).to_series())

retrain_y_arr = retrain_y.drop(CONFIG.DATE_COL).to_numpy()
retrain_y = pl.DataFrame(gaussian_rank_transform(retrain_y_arr), schema=train_y.drop(CONFIG.DATE_COL).columns).insert_column(
    0, train_y.select(CONFIG.DATE_COL).to_series()
)


# pl.DataFrame(
#     (retrain_y_arr - np.nanmean(retrain_y_arr, axis=1).reshape(retrain_y_arr.shape[0], -1))
#     / np.nanstd(retrain_y_arr, axis=1).reshape(retrain_y_arr.shape[0], -1),
#     schema=train_y.drop(CONFIG.DATE_COL).columns,
# ).insert_column(0, train_y.select(CONFIG.DATE_COL).to_series())


create_market_features took 16.2510 seconds


In [19]:
s = FeatureSelector(train_x=train_x.drop(CONFIG.DATE_COL), train_y=train_y.drop(CONFIG.DATE_COL))
corr, bi_corr, keep_features, binary_features = s.run_selection()

run_correlation took 31.2631 seconds
run_biserial_correlation took 0.0785 seconds
After correlation, keeping: 1393 features


In [20]:
corr_df = pl.concat(
    [
        pl.concat(
            [
                pl.DataFrame(corr.mean(axis=1), schema=["corr_mean"]).with_columns(
                    pl.Series(name="feature", values=s.train_x.drop(binary_features).columns)
                ),
                pl.DataFrame(corr.std(axis=1), schema=["corr_std"]),
            ],
            how="horizontal",
        ),
        pl.concat(
            [
                pl.DataFrame(bi_corr.mean(axis=1), schema=["corr_mean"]).with_columns(pl.Series(name="feature", values=binary_features)),
                pl.DataFrame(bi_corr.std(axis=1), schema=["corr_std"]),
            ],
            how="horizontal",
        ),
    ],
    how="vertical",
).sort(by="corr_mean")


In [21]:
keep = []
keep_df = {}
for i in range(len(CONFIG.LAG_SEQ_LEN.keys())):
    score = corr[:, i : (i + 1) * 106]
    bi_score = bi_corr[:, i : (i + 1) * 106]
    df = pl.concat(
        [
            pl.concat(
                [
                    pl.DataFrame(score.mean(axis=1), schema=["corr_mean"]).with_columns(
                        pl.Series(name="feature", values=s.train_x.drop(binary_features).columns)
                    ),
                    pl.DataFrame(score.std(axis=1), schema=["corr_std"]),
                    pl.DataFrame(np.median(score, axis=1), schema=["corr_median"]),
                ],
                how="horizontal",
            ),
            pl.concat(
                [
                    pl.DataFrame(bi_score.mean(axis=1), schema=["corr_mean"]).with_columns(pl.Series(name="feature", values=binary_features)),
                    pl.DataFrame(bi_score.std(axis=1), schema=["corr_std"]),
                    pl.DataFrame(np.median(bi_score, axis=1), schema=["corr_median"]),
                ],
                how="horizontal",
            ),
        ],
        how="vertical",
    )

    # Normalize the statistics columns for later use in composite scoring
    df = df.with_columns(
        ((pl.col("corr_mean") - df["corr_mean"].min()) / (df["corr_mean"].max() - df["corr_mean"].min())).alias("corr_mean_norm"),
        ((pl.col("corr_std") - df["corr_std"].min()) / (df["corr_std"].max() - df["corr_std"].min())).alias("corr_std_norm"),
        ((pl.col("corr_median") - df["corr_median"].min()) / (df["corr_median"].max() - df["corr_median"].min())).alias("corr_median_norm"),
    )

    # Compute the composite score using weighted sums of normalized mean, std, and median
    df = df.with_columns((0.2 * pl.col("corr_mean_norm") + 0.6 * pl.col("corr_std_norm") + 0.2 * pl.col("corr_median_norm")).alias("composite_score"))

    # Sort by composite score and select top 50 features
    df_sorted = df.sort(by="composite_score", descending=True).head(100)
    # Select features from sorted list
    keep.append(df_sorted.select("feature").to_series().to_list())

In [ ]:
keep = []
keep_df = {}
for i in range(len(CONFIG.LAG_SEQ_LEN.keys())):
    score = corr[:, i : (i + 1) * 106]
    df = pl.concat(
        [
            pl.DataFrame(score.mean(axis=1), schema=["corr_mean"]).with_columns(pl.Series(name="feature", values=s.train_x.columns)),
            pl.DataFrame(score.std(axis=1), schema=["corr_std"]),
        ],
        how="horizontal",
    )

    filter_mean = df.sort(by="corr_mean", descending=True).head(500)
    filter_features = filter_mean.select("feature").to_series().to_list()

    train = train_x.select(filter_features)
    X_train, y_train, X_val, y_val = (
        train[: int(0.8 * len(train))],
        train_y.select([f"target_{i}" for i in range(i, (i + 1) * 106)])[: int(0.8 * len(train))],
        train[int(0.8 * len(train)) + 1 :],
        train_y.select([f"target_{i}" for i in range(i, (i + 1) * 106)])[int(0.8 * len(train)) + 1 :],
    )

    model = xgb.XGBRegressor(
        **{
            "n_estimators": 100,
            "max_depth": 32,
            "learning_rate": 0.05,
            "subsample": 0.7,
            "colsample_bytree": 0.5,
            "random_state": CONFIG.RANDOM_STATE,
            "device": "gpu",
            "early_stopping_rounds": 10,
            "verbosity": 2,
        }
    )
    model.fit(
        X_train.to_pandas(),
        y_train.fill_nan(-999).to_pandas(),
        eval_set=[(X_val.to_pandas(), y_val.fill_nan(-999).to_pandas())],
        verbose=True,
    )

    raw_score = model.get_booster().get_score(importance_type="gain")
    importance_dict = {f: raw_score.get(f, 0.0) for f in X_train.columns}
    feat_impt_xgb = (
        pl.DataFrame(importance_dict)
        .transpose(include_header=True)
        .with_columns(pl.col("column_0") / pl.col("column_0").sum())
        .sort(by="column_0", descending=True)
    )
    feat_impt_xgb.columns = ["feature", "score"]
    final_scores = (
        feat_impt_xgb.join(filter_mean, on="feature")
        .with_columns(total_score=(pl.col("corr_mean") * 0.5 + pl.col("score") * 0.5))
        .sort(by="total_score", descending=True)
    )

    feats = final_scores.select("feature").to_series().head(50).to_list()
    keep.append(feats)
    keep_df[i] = final_scores

In [22]:
list(set([j for i in keep for j in i])).__len__()

270

In [23]:
list(set([j for i in keep for j in i]))

['US_Stock_DE_adj_low_std_90_252',
 'target_96_return_lag_4_std_252_252',
 'US_Stock_SHEL_adj_low_vol_90_std_10_90',
 'target_251_auto_corr_90_std_252_90',
 'US_Stock_JNK_adj_close_log_ret_return_skew_10_std_252_90',
 'target_63_return_lag_4_std_10_252',
 'US_Stock_CLF_adj_open_vol_10_std_252_10',
 'target_381_return_lag_1_std_90_90',
 'target_385_return_skew_5_std_252_10',
 'US_Stock_DE_adj_open_10_sortino_std_10_252',
 'target_96_return_lag_4_std_10_252',
 'LME_ZS_Close_log_ret_US_Stock_MS_adj_close_log_ret_sigmoid_diff_std_90_252',
 'US_Stock_JNK_adj_close_log_ret_return_skew_252_std_10_252',
 'FX_ZAREUR_10_sortino_std_10_10',
 'LME_ZS_Close_log_ret_FX_NOKGBP_log_ret_log_ratio_std_90_10',
 'US_Stock_CVX_adj_close_std_252_10',
 'US_Stock_DE_adj_close_sma_10_std_90_252',
 'US_Stock_ENB_adj_close_log_ret_return_lag_5_std_90_252',
 'target_1_return_lag_2_std_90_10',
 'target_184_return_skew_252_std_90_90',
 'LME_ZS_Close_log_ret_US_Stock_MS_adj_close_log_ret_sigmoid_diff_std_252_90',
 '